<img src='https://kabardewata.com/uploads/image/news_agustus/bumbu-bali-restaurant.jpg' style='width:100%; position:absolute;'>
<h1 style='text-align:center; font-size:100px; position:relative; opacity:0.9; padding-top:200px;'>THE BATTLE OF NEIGHBORHOODS</h1>